# Práctica 06: Redes bayesianas

### Descripción del Algoritmo

Las redes bayesianas son modelos gráficos probabilísticos que representan un conjunto de variables y sus dependencias condicionales mediante un grafo dirigido acíclico (DAG). Cada nodo en el grafo representa una variable aleatoria, mientras que cada arista representa una dependencia condicional entre las variables. Este modelo es útil para realizar inferencias sobre variables desconocidas, dadas las observaciones de otras variables.

Las redes bayesianas utilizan el teorema de Bayes para actualizar las probabilidades de los nodos a medida que se dispone de nueva información, permitiendo realizar predicciones y entender la relación entre diferentes variables en un sistema complejo.

#### Función y Parámetros

##### Función: Creación de la Red Bayesiana

- **Nombre**: `BayesianModel()`
    - **Descripción**: Crea una instancia de una red bayesiana. Define la estructura del modelo especificando las dependencias entre las variables.
    - **Parámetros**:
        - `edges`: Una lista de tuplas que representan las aristas del grafo. Cada tupla (`(A, B)`) indica una dependencia de `B` sobre `A`, es decir, que `A` es padre de `B`.
    - **Ejemplo de Uso**:
        ```python
        modelo = BayesianModel([
            ('Horas de trabajo', 'Balance trabajo-vida'),
            ('Balance trabajo-vida', 'Satisfacción laboral')
        ])
        ```

##### Función: Definición de las Tablas de Probabilidad Condicional (CPTs)

- **Nombre**: `TabularCPD()`
    - **Descripción**: Define una tabla de probabilidad condicional (CPT) para una variable dentro de la red. La CPT cuantifica el efecto de los nodos padres sobre el nodo hijo.
    - **Parámetros**:
        - `variable`: El nombre de la variable para la CPT.
        - `variable_card`: El número de estados posibles de la variable.
        - `values`: Una matriz de probabilidades que define la distribución condicional de la variable dado el estado de sus padres.
        - `evidence` (opcional): La lista de variables padres.
        - `evidence_card` (opcional): El número de estados posibles para cada variable padre.
    - **Ejemplo de Uso**:
        ```python
        cpt_h = TabularCPD(variable='Horas de trabajo', variable_card=3,
                           values=[[0.1], [0.6], [0.3]],  
                           state_names={'Horas de trabajo': ['largas', 'moderadas', 'cortas']})
        ```

##### Función: Verificación del modelo de la red Bayesiana

- **Nombre**: `check_model()`
    - **Descripción**: Este método verifica el modelo de la red bayesiana para detectar errores. Verifica si la suma de las probabilidades para cada estado es igual a 1 y si las CPDs asociadas con los nodos son consistentes con sus padres.
    - **Parámetros**: Ninguno.
    - **Retorno**:
        - `check` (booleano): True si todas las verificaciones pasan, de lo contrario debería lanzar un error..
    - **Ejemplo de Uso**:
        ```python
        if modelo.check_model():
            print("El modelo es válido.")
        else:
            print("El modelo es inválido.")
        ```

##### Función: Realización de Inferencias

- **Nombre**: `VariableElimination()`
    - **Descripción**: Este método de inferencia permite consultar la red bayesiana para obtener las probabilidades condicionales de una o más variables, dadas algunas evidencias.
    - **Parámetros**:
        - `model`: La red bayesiana sobre la cual se realiza la inferencia.
    - **Ejemplo de Uso**:
        ```python
        inferencia = VariableElimination(modelo)
        resultado = inferencia.query(variables=['Satisfacción laboral'],
                                     evidence={'Horas de trabajo': 'moderadas', 'Balance trabajo-vida': 'equilibrado'})

        print(resultado)
        ```


<center><h1>INICIO DEL CÓDIGO</h1></center>





In [2]:
# Importar las librerías necesarias
from pgmpy.models import BayesianNetwork # Se usa BayesianModel en lugar de BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

# Paso 1: Definir la estructura de la red bayesiana
# Aquí se establece cómo se relacionan las variables dentro de la red.
# Las aristas representan influencias directas entre las variables, modelando las dependencias condicionales.
modelo = BayesianNetwork([
    ('Horas de trabajo', 'Balance trabajo-vida'),  # Las horas de trabajo influyen en el balance trabajo-vida
    ('Balance trabajo-vida', 'Satisfacción laboral')  # El balance trabajo-vida influye en la satisfacción laboral
])

# Paso 2: Definir las CPTs (Tablas de Probabilidad Condicional)
# Cada CPT especifica cómo la probabilidad de una variable se ve influenciada por el estado de sus padres en la red.

# CPT para Horas de trabajo (H)
# Define la distribución inicial de cómo se distribuyen las horas de trabajo sin considerar otras variables.
cpt_h = TabularCPD(variable='Horas de trabajo', variable_card=3,
                   values=[[0.6], [0.3], [0.1]],  # Suposiciones de ejemplo sobre la distribución
                   state_names={'Horas de trabajo': ['largas', 'moderadas', 'cortas']})

# CPT para Balance trabajo-vida (B), dependiente de H
# Establece cómo las horas de trabajo afectan el balance trabajo-vida.
cpt_b = TabularCPD(variable='Balance trabajo-vida', variable_card=2,
                   values=[[0.4, 0.7, 0.2], [0.6, 0.3, 0.8]],  # Probabilidades condicionales basadas en H
                   evidence=['Horas de trabajo'], evidence_card=[3],
                   state_names={'Balance trabajo-vida': ['equilibrado', 'no equilibrado'],
                                'Horas de trabajo': ['largas', 'moderadas', 'cortas']})

# CPT para Satisfacción laboral (S), dependiente de B
# Define cómo el balance trabajo-vida afecta la satisfacción laboral.
cpt_s = TabularCPD(variable='Satisfacción laboral', variable_card=3,
                   values=[[0.8, 0.05], [0.15, 0.6], [0.05, 0.35]],  # Probabilidades condicionales basadas en B
                   evidence=['Balance trabajo-vida'], evidence_card=[2],
                   state_names={'Satisfacción laboral': ['satisfecho', 'neutral', 'insatisfecho'],
                                'Balance trabajo-vida': ['equilibrado', 'no equilibrado']})

# Añadir las CPTs al modelo y verificar su validez
modelo.add_cpds(cpt_h, cpt_b, cpt_s)

# check_model() lo usamos para verificar si un modelo bayesiado es valido, para esto necesitamos asegurarnos de que las CPDs(las tablas de probabilidad
# condicional que en nuestro caso son 3 variables) esten correctamente definidas para que la rede bayesiana que se creo tenga coherencia y sentido, para asi poder obtener
# resultados validos.
#
# Al realizar check_model() si el modelo que se definio es valido, entonces esto nos indica que la relaciones entre las 3 variables de la red bayesiana las hemos
# unido correctamente, y entonces regresa true con el mensaje "El modelo es valido", en caso contrario regresamos false y decimos "El modelos es inválido", esto nos dice que
# lo mas seguro es que las relaciones de la red bayesiana que se definieron se cometieron errores dando lugar a una inconsistencia en las tabalas de probabilidad.
if modelo.check_model():
    print("El modelo es válido.")
else:
    print("El modelo es inválido.")

# Paso 3: Realizar la inferencia
# Se utiliza VariableElimination para consultar la red y realizar inferencias sobre variables de interés.

# Crear un objeto de inferencia
inferencia = VariableElimination(modelo)

# Consultar la probabilidad de estar satisfecho con el trabajo bajo condiciones específicas.
# Esto permite entender cómo factores como las horas de trabajo y el balance trabajo-vida afectan la satisfacción laboral.
print("La probabilidad de estar satisfecho es:")
resultado = inferencia.query(variables=['Satisfacción laboral'],
                             evidence={'Horas de trabajo': 'moderadas', 'Balance trabajo-vida': 'equilibrado'})

print(resultado)


El modelo es válido.
La probabilidad de estar satisfecho es:
+------------------------------------+-----------------------------+
| Satisfacción laboral               |   phi(Satisfacción laboral) |
+====================================+=============================+
| Satisfacción laboral(satisfecho)   |                      0.8000 |
+------------------------------------+-----------------------------+
| Satisfacción laboral(neutral)      |                      0.1500 |
+------------------------------------+-----------------------------+
| Satisfacción laboral(insatisfecho) |                      0.0500 |
+------------------------------------+-----------------------------+
